In [1]:
import onnx 
from onnx import shape_inference
import onnxruntime  as rt
import pickle

In [2]:
model =  onnx.load("ten_vad_exported.onnx")
model = shape_inference.infer_shapes(model)

In [3]:
new_outputs = []
for val_info in model.graph.value_info:
    new_outputs.append(val_info)
    
for o in new_outputs:
    model.graph.output.append(o)

onnx.save(model, "debug_torch_model.onnx")


In [5]:
orig_model =  onnx.load("src/onnx_model/ten-vad.onnx")
orig_model = shape_inference.infer_shapes(orig_model)
new_outputs = []
for val_info in orig_model.graph.value_info:
    new_outputs.append(val_info)
    
for o in new_outputs:
    orig_model.graph.output.append(o)

onnx.save(orig_model, "debug_onnx_model.onnx")


In [9]:
model_name = "debug_torch_model.onnx"
model = onnx.load(model_name)
with open("pickle_input.pkl", "rb") as f:
    input_t = pickle.load(f)

session = rt.InferenceSession(model_name, providers=['CPUExecutionProvider'])
output_names = [ o.name for o in session.get_outputs()]
raw_outputs = session.run(None, input_t)
outputs_dict = dict(zip(output_names, raw_outputs))
outputs_dict.keys()


dict_keys(['output', '135', '136', '196', '197', '/Unsqueeze_output_0', '/Constant_output_0', '/Reshape_output_0', '/conv_dw/Conv_output_0', '/conv_pw/Conv_output_0', '/relu/Relu_output_0', '/pool/MaxPool_output_0', '/sep1_dw/Conv_output_0', '/sep1_pw/Conv_output_0', '/Constant_1_output_0', '/Shape_output_0', '/Gather_output_0', '/Constant_2_output_0', '/Equal_output_0', '/If_output_0', '/relu_1/Relu_output_0', '/Unsqueeze_1_output_0', '/sep2_dw/Conv_output_0', '/sep2_pw/Conv_output_0', '/Constant_3_output_0', '/Shape_1_output_0', '/Gather_1_output_0', '/Constant_4_output_0', '/Equal_1_output_0', '/If_1_output_0', '/relu_2/Relu_output_0', '/Transpose_output_0', '/Shape_2_output_0', '/Constant_5_output_0', '/Gather_2_output_0', '/Shape_3_output_0', '/Constant_6_output_0', '/Gather_3_output_0', '/Shape_4_output_0', '/Constant_7_output_0', '/Gather_4_output_0', '/Mul_output_0', '/Unsqueeze_2_output_0', '/Constant_8_output_0', '/Unsqueeze_3_output_0', '/Concat_output_0', '/Reshape_1_output

In [8]:
model_name = "debug_onnx_model.onnx"
model = onnx.load(model_name)
with open("pickle_input.pkl", "rb") as f:
    input_t = pickle.load(f)

session2 = rt.InferenceSession(model_name, providers=['CPUExecutionProvider'])
output_names2 = [ o.name for o in session2.get_outputs()]
raw_outputs2 = session2.run(None, input_t)
outputs_dict2 = dict(zip(output_names2, raw_outputs2))
outputs_dict2.keys()

dict_keys(['output_1', 'output_2', 'output_3', 'output_6', 'output_7', 'Unsqueeze__97:0', 'Unsqueeze__95:0', 'Unsqueeze__68:0', 'Unsqueeze__66:0', 'StatefulPartitionedCall/vad_model/ExpandDims:0', 'StatefulPartitionedCall/vad_model/separable_conv2d/separable_conv2d/depthwise__111:0', 'StatefulPartitionedCall/vad_model/separable_conv2d/separable_conv2d/depthwise:0', 'StatefulPartitionedCall/vad_model/separable_conv2d/BiasAdd:0', 'StatefulPartitionedCall/vad_model/separable_conv2d/Relu:0', 'StatefulPartitionedCall/vad_model/separable_conv1d/separable_conv2d/depthwise__127:0', 'StatefulPartitionedCall/vad_model/separable_conv1d/separable_conv2d/depthwise:0', 'StatefulPartitionedCall/vad_model/separable_conv1d/BiasAdd:0', 'StatefulPartitionedCall/vad_model/separable_conv1d/Squeeze:0', 'StatefulPartitionedCall/vad_model/separable_conv1d/Relu:0', 'StatefulPartitionedCall/vad_model/separable_conv1d_1/ExpandDims:0', 'StatefulPartitionedCall/vad_model/separable_conv1d_1/separable_conv2d/depthwi

In [11]:
onnx_ouput=outputs_dict2['StatefulPartitionedCall/vad_model/separable_conv2d/separable_conv2d/depthwise:0'].shape
torch_output=outputs_dict['/conv_dw/Conv_output_0'].shape


(1, 1, 1, 39)